In [1]:
import tensorflow as tf
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 设置随机种子以确保结果可重复
tf.random.set_seed(100)
np.random.seed(100)

# 读取数据
# 请确保 '数据融合--高温合金数据库.xlsx' 文件路径正确
data = pd.read_excel('数据融合--高温合金数据库.xlsx')

# 提取特征和标签
# 假设 'UTS', 'EL', 'TYS', 'RA' 是目标变量
features = data.drop(['UTS', 'EL', 'TYS', 'RA', '合金牌号', '材料', '熔炼工艺'], axis=1)
labels = data[['UTS', 'EL', 'TYS', 'RA']].values

# 对特征进行独热编码（如果有分类变量）
features = pd.get_dummies(features)

# 转换为 NumPy 数组
features = features.values

# 归一化处理
scaler = MinMaxScaler()
features = scaler.fit_transform(features)

# 划分训练集、验证集和测试集
test_ratio = 0.25
validation_ratio = 0.2  # 20% 的数据用于验证集合

# 首先划分训练集和临时集
train_features, temp_features, train_labels, temp_labels = train_test_split(
    features, labels, test_size=(test_ratio + validation_ratio), random_state=100)

# 再将临时集划分为测试集和验证集
test_features, validation_features, test_labels, validation_labels = train_test_split(
    temp_features, temp_labels, 
    test_size=validation_ratio / (test_ratio + validation_ratio),
    random_state=100)

# 构建多目标回归神经网络模型
model_bpnn = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(train_features.shape[1],)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(labels.shape[1])  # 输出层神经元数等于目标变量数
])

# 编译模型
model_bpnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

# 打印模型架构
model_bpnn.summary()

# 定义早停回调以防止过拟合
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 训练模型
history = model_bpnn.fit(
    train_features, train_labels,
    epochs=100,
    batch_size=32,
    validation_data=(validation_features, validation_labels),
    callbacks=[early_stop],
    verbose=1
)

# 预测结果
train_predicted = model_bpnn.predict(train_features)
test_predicted = model_bpnn.predict(test_features)
validation_predicted = model_bpnn.predict(validation_features)

# 计算性能指标
def compute_metrics(true, predicted):
    rmse = np.sqrt(mean_squared_error(true, predicted, multioutput='raw_values'))
    r2 = r2_score(true, predicted, multioutput='raw_values')
    return rmse, r2

# 定义目标变量列表
target_names = ['UTS', 'EL', 'TYS', 'RA']
metrics = {}

for i, target in enumerate(target_names):
    metrics[target] = {}
    # 训练集
    rmse_tr, r2_tr = compute_metrics(train_labels[:, i], train_predicted[:, i])
    metrics[target]['Train_RMSE'] = rmse_tr
    metrics[target]['Train_R2'] = r2_tr
    # 测试集
    rmse_te, r2_te = compute_metrics(test_labels[:, i], test_predicted[:, i])
    metrics[target]['Test_RMSE'] = rmse_te
    metrics[target]['Test_R2'] = r2_te
    # 验证集
    rmse_val, r2_val = compute_metrics(validation_labels[:, i], validation_predicted[:, i])
    metrics[target]['Validation_RMSE'] = rmse_val
    metrics[target]['Validation_R2'] = r2_val

# 输出性能指标
for target in target_names:
    print(f"=== {target} Performance ===")
    print(f"RMSE (Train): {metrics[target]['Train_RMSE']:.2f}")
    print(f"RMSE (Test): {metrics[target]['Test_RMSE']:.2f}")
    print(f"RMSE (Validation): {metrics[target]['Validation_RMSE']:.2f}")
    print(f"R^2 (Train): {metrics[target]['Train_R2']:.2f}")
    print(f"R^2 (Test): {metrics[target]['Test_R2']:.2f}")
    print(f"R^2 (Validation): {metrics[target]['Validation_R2']:.2f}\n")

# 绘制训练过程中的损失函数变化
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Train Loss', color='blue')
plt.plot(history.history['val_loss'], label='Validation Loss', color='orange')
plt.title('Model Loss During Training')
plt.xlabel('Epochs')
plt.ylabel('Mean Squared Error')
plt.legend()
plt.show()

# 设置绘图参数
plt.rcParams['figure.figsize'] = (20, 15)
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.dpi'] = 300

# 创建子图
fig, axes = plt.subplots(2, 2, figsize=(20, 15))
axes = axes.flatten()

for i, target in enumerate(target_names):
    ax = axes[i]
    ax.set_facecolor("white")
    ax.grid(True, linestyle=':', linewidth=0.9, alpha=0.8)
    for spine in ax.spines.values():
        spine.set_linewidth(1.5)
        spine.set_color('black')
    
    # 确定坐标轴范围
    max_val = max(
        np.max(labels[:, i]),
        np.max([train_predicted[:, i], test_predicted[:, i], validation_predicted[:, i]])) * 1.05
    ax.set_xticks(np.linspace(0, max_val, 11))
    ax.set_yticks(np.linspace(0, max_val, 11))
    
    # 绘制散点
    ax.scatter(train_labels[:, i], train_predicted[:, i], alpha=0.7, s=33, linewidths=0.4, c='#85a2d2',
               edgecolor='#000000', label=f'Training set (R²={metrics[target]["Train_R2"]:.2f})', marker='o')
    ax.scatter(test_labels[:, i], test_predicted[:, i], alpha=0.7, s=33, linewidths=0.4, c='#c2acda',
               edgecolor='#000000', label=f'Testing set (R²={metrics[target]["Test_R2"]:.2f})', marker='o')
    ax.scatter(validation_labels[:, i], validation_predicted[:, i], alpha=0.7, s=33, linewidths=0.4, c='#c0db82',
               edgecolor='#000000', label=f'Validation set (R²={metrics[target]["Validation_R2"]:.2f})', marker='o')
    
    # 绘制参考线
    ax.plot([0, max_val], [0, max_val], 'b--', linewidth=0.8, alpha=0.8)
    ax.set_xlim(0, max_val)
    ax.set_ylim(0, max_val)
    
    # 设置标签和标题
    ax.set_xlabel(f'Real {target}', fontsize=16, weight='bold', labelpad=20, fontname='Times New Roman')
    ax.set_ylabel(f'Predicted {target}', fontsize=16, weight='bold', labelpad=20, fontname='Times New Roman')
    ax.set_title(f'{target}: Real vs Predicted', fontsize=18, weight='bold', fontname='Times New Roman')
    
    # 设置图例
    ax.legend(prop={'family': 'Times New Roman', 'weight': 'bold', 'size': 12}, loc='upper left')

# 添加整体标题
plt.suptitle('Multi-Objective Neural Network Regressor: Real vs Predicted for UTS, EL, TYS, and RA', fontsize=22, weight='bold', fontname='Times New Roman')

# 调整布局
plt.tight_layout(rect=[0, 0.03, 1, 0.95])

# 显示图形
plt.show()

2024-12-25 11:06:55.310999: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/macbookpro/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/macbookpro/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               2176      
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 4)                 516       
                                                                 
Total params: 19204 (75.02 KB)
Trainable params: 19204 (75.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
18/18 [==============================] - 3s 48ms/step - loss: 414793.9375 - mean_squared_error: 414793.9375 - val_loss: 404107.4062 - val_mean_squared_error: 404107.4062
Epoch 2/100
18/18 [==============================] - 0s 7ms/step - loss: 412086.9688 - mean_squared_er

18/18 [==============================] - 0s 4ms/step - loss: 30574.7168 - mean_squared_error: 30574.7168 - val_loss: 30551.3457 - val_mean_squared_error: 30551.3457
Epoch 43/100
18/18 [==============================] - 0s 4ms/step - loss: 30048.9082 - mean_squared_error: 30048.9082 - val_loss: 30110.3379 - val_mean_squared_error: 30110.3379
Epoch 44/100
18/18 [==============================] - 0s 4ms/step - loss: 29517.3203 - mean_squared_error: 29517.3203 - val_loss: 29610.7441 - val_mean_squared_error: 29610.7441
Epoch 45/100
18/18 [==============================] - 0s 4ms/step - loss: 28958.6699 - mean_squared_error: 28958.6699 - val_loss: 29270.3359 - val_mean_squared_error: 29270.3359
Epoch 46/100
18/18 [==============================] - 0s 4ms/step - loss: 28494.0898 - mean_squared_error: 28494.0898 - val_loss: 28841.4238 - val_mean_squared_error: 28841.4238
Epoch 47/100
18/18 [==============================] - 0s 4ms/step - loss: 28068.2168 - mean_squared_error: 28068.2168 - val

18/18 [==============================] - 0s 4ms/step - loss: 22415.6836 - mean_squared_error: 22415.6836 - val_loss: 24535.3535 - val_mean_squared_error: 24535.3535
Epoch 89/100
18/18 [==============================] - 0s 4ms/step - loss: 22236.6191 - mean_squared_error: 22236.6191 - val_loss: 24396.7090 - val_mean_squared_error: 24396.7090
Epoch 90/100
18/18 [==============================] - 0s 4ms/step - loss: 22164.3535 - mean_squared_error: 22164.3535 - val_loss: 24637.0840 - val_mean_squared_error: 24637.0840
Epoch 91/100
18/18 [==============================] - 0s 4ms/step - loss: 22054.0488 - mean_squared_error: 22054.0488 - val_loss: 24414.0957 - val_mean_squared_error: 24414.0957
Epoch 92/100
18/18 [==============================] - 0s 4ms/step - loss: 22017.5723 - mean_squared_error: 22017.5723 - val_loss: 24420.3281 - val_mean_squared_error: 24420.3281
Epoch 93/100
18/18 [==============================] - 0s 4ms/step - loss: 21982.1543 - mean_squared_error: 21982.1543 - val

TypeError: unsupported format string passed to numpy.ndarray.__format__